In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-07 23:17:46.470558: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 23:17:46.470634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 23:17:46.474152: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 23:17:46.495024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 23:17:49.113031: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../cell_images/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/27558
[INFO]: Processed 2000/27558
[INFO]: Processed 3000/27558
[INFO]: Processed 4000/27558
[INFO]: Processed 5000/27558
[INFO]: Processed 6000/27558
[INFO]: Processed 7000/27558
[INFO]: Processed 8000/27558
[INFO]: Processed 9000/27558
[INFO]: Processed 10000/27558
[INFO]: Processed 11000/27558
[INFO]: Processed 12000/27558
[INFO]: Processed 13000/27558
[INFO]: Processed 14000/27558
[INFO]: Processed 15000/27558
[INFO]: Processed 16000/27558
[INFO]: Processed 17000/27558
[INFO]: Processed 18000/27558
[INFO]: Processed 19000/27558
[INFO]: Processed 20000/27558
[INFO]: Processed 21000/27558
[INFO]: Processed 22000/27558
[INFO]: Processed 23000/27558
[INFO]: Processed 24000/27558
[INFO]: Processed 25000/27558
[INFO]: Processed 26000/27558
[INFO]: Processed 27000/27558
(27558, 224, 224, 3)
(27558,)


In [8]:
from tensorflow.keras.applications import  ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
with tf.device('/CPU:0'):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
    model = Model(inputs=base_model.input, outputs=base_model.output)
    features = model.predict(data, batch_size=32, verbose=1)

# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-07 23:18:54.345439: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38380 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0


862/862 [==============================] - 1703s 2s/step
(27558, 100352)


In [9]:
data = features_flatten

In [10]:
import gc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
#tf.config.experimental.enable_tensor_float_32_execution(True)
#tf.config.threading.set_inter_op_parallelism_threads(1)
#tf.config.threading.set_intra_op_parallelism_threads(1)
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16":
        labels = to_categorical(labels)

    elif name == "VGG19":  
        labels = to_categorical(labels)
    else:
        data = data
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "VGG19":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [11]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9390420899854862
Sensitivity: 0.9375907111756169
Specificity: 0.9404934687953556
AUC-ROC: 0.9390420899854861
MCC: 0.8780878793637907
Precision: 0.9403202328966521
F1 Score: 0.938953488372093

[INFO] Fold 2 / 10 for LR
Accuracy: 0.9470246734397678
Sensitivity: 0.951378809869376
Specificity: 0.9426705370101597
AUC-ROC: 0.9470246734397678
MCC: 0.894083248484042
Precision: 0.9431654676258993
F1 Score: 0.9472543352601156

[INFO] Fold 3 / 10 for LR
Accuracy: 0.9412191582002902
Sensitivity: 0.9448476052249637
Specificity: 0.9375907111756169
AUC-ROC: 0.9412191582002903
MCC: 0.8824615530272384
Precision: 0.9380403458213257
F1 Score: 0.9414316702819957

[INFO] Fold 4 / 10 for LR
Accuracy: 0.9444847605224964
Sensitivity: 0.9477503628447025
Specificity: 0.9412191582002902
AUC-ROC: 0.9444847605224963
MCC: 0.8889884818753946
Precision: 0.9416005767844268
F1 Score: 0.9446654611211572

[INFO] Fold 5 / 10 for LR
Accuracy: 0.9299709724238027
Sensitivity: 0.9317851959361393
Specificity: 0.928

In [12]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.8102322206095791
Sensitivity: 0.8403483309143687
Specificity: 0.7801161103047896
AUC-ROC: 0.8102322206095791
MCC: 0.6215930107212115
Precision: 0.7926078028747433
F1 Score: 0.8157802042972878

[INFO] Fold 2 / 10 for NB
Accuracy: 0.8134978229317852
Sensitivity: 0.8352685050798259
Specificity: 0.7917271407837445
AUC-ROC: 0.8134978229317852
MCC: 0.627590837267297
Precision: 0.8004172461752433
F1 Score: 0.8174715909090908

[INFO] Fold 3 / 10 for NB
Accuracy: 0.8200290275761973
Sensitivity: 0.8563134978229318
Specificity: 0.783744557329463
AUC-ROC: 0.8200290275761974
MCC: 0.6417500943467965
Precision: 0.7983761840324763
F1 Score: 0.8263305322128853

[INFO] Fold 4 / 10 for NB
Accuracy: 0.8167634252539913
Sensitivity: 0.8497822931785196
Specificity: 0.783744557329463
AUC-ROC: 0.8167634252539913
MCC: 0.6349127849578589
Precision: 0.7971409121851599
F1 Score: 0.8226203020723568

[INFO] Fold 5 / 10 for NB
Accuracy: 0.816400580551524
Sensitivity: 0.841799709

In [13]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.7235123367198839
Sensitivity: 0.9731494920174165
Specificity: 0.47387518142235124
AUC-ROC: 0.7235123367198839
MCC: 0.5159302690657588
Precision: 0.649080348499516
F1 Score: 0.7787456445993032

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.7264150943396226
Sensitivity: 0.9709724238026125
Specificity: 0.4818577648766328
AUC-ROC: 0.7264150943396226
MCC: 0.5191699914900667
Precision: 0.652046783625731
F1 Score: 0.7801749271137026

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.7343976777939042
Sensitivity: 0.9775036284470247
Specificity: 0.49129172714078373
AUC-ROC: 0.7343976777939042
MCC: 0.5364767597351032
Precision: 0.65771484375
F1 Score: 0.7863397548161121

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.7187953555878084
Sensitivity: 0.9702467343976778
Specificity: 0.46734397677793904
AUC-ROC: 0.7187953555878084
MCC: 0.5062697479116127
Precision: 0.6455818445195558
F1 Score: 0.775297187590606

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.7362119013062409
Sensitivity: 0.96444

In [14]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.897677793904209
Sensitivity: 0.8613933236574746
Specificity: 0.9339622641509434
AUC-ROC: 0.8976777939042091
MCC: 0.7974581671248175
Precision: 0.9287949921752738
F1 Score: 0.8938253012048192

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.8991291727140783
Sensitivity: 0.8679245283018868
Specificity: 0.93033381712627
AUC-ROC: 0.8991291727140784
MCC: 0.7998174773597547
Precision: 0.9256965944272446
F1 Score: 0.8958801498127341

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.9125544267053701
Sensitivity: 0.8925979680696662
Specificity: 0.9325108853410741
AUC-ROC: 0.91255442670537
MCC: 0.8257668557858249
Precision: 0.9297052154195011
F1 Score: 0.9107737874861164

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.9107402031930334
Sensitivity: 0.8831640058055152
Specificity: 0.9383164005805515
AUC-ROC: 0.9107402031930334
MCC: 0.8227326479665596
Precision: 0.934715821812596
F1 Score: 0.9082089552238805

[INFO] Fold 5 / 10 for Random Forest


In [15]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 12401, number of negative: 12401
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 48.738065 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20004802
[LightGBM] [Info] Number of data points in the train set: 24802, number of used features: 95357
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Accuracy: 0.9386792452830188
Sensitivity: 0.9412191582002902
Specificity: 0.9361393323657474
AUC-ROC: 0.9386792452830187
MCC: 0.8773698107409595
Precision: 0.9364620938628159
F1 Score: 0.9388346000723851

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Number of positive: 12401, number of negative: 12401
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 48.438812 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20009063
[LightGBM] [Info] N

In [16]:
# Save results to a CSV file
results_df.to_csv('DS1_ResNet50.csv', index=False)

In [17]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.941433,0.942594,0.940271,0.941433,0.882895,0.940416,0.941490,6615.441406,9456.957862
1,NB,0.814029,0.839975,0.788083,0.814029,0.628960,0.798547,0.818707,10549.937500,558.072101
2,KNN,0.726541,0.970318,0.482765,0.726541,0.518942,0.652341,0.780158,10560.718750,1489.918523
3,Random Forest,0.900683,0.870456,0.930908,0.900682,0.802894,0.926448,0.897535,10647.679688,1393.638304
4,LGBM,0.937587,0.937006,0.938166,0.937586,0.875246,0.938127,0.937528,12868.769531,17377.519214
